# Visualizing Data with Pandas, Matplotlib, and Seaborn

[Matplotlib](http://matplotlib.org) is basic plotting library for Python inspired by Matlab. 
[Seaborn](http://stanford.edu/~mwaskom/software/seaborn) is built on top of matplotlib with integrated analysis and specialized plots.

Also see [the full gallery of Seaborn](http://stanford.edu/~mwaskom/software/seaborn/examples/index.html) or [Matplotlib](http://matplotlib.org/gallery.html).

We're going to be using data on football player attributes from FIFA 2017. Why? Well, because it's fun, it's highly multi-dimensional, and we can process the data in an infinite number of ways. The [dataset](https://www.kaggle.com/artimous/complete-fifa-2017-player-dataset-global) has some other parts too, in case you're interested in continuing poking around this data. 


In [ ]:
#disable some annoying warning
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

#plots the figures in place instead of a new window
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
from pandas.tools.plotting import radviz, parallel_coordinates, andrews_curves
import numpy as np

## Load Data

Here we're going to use the gapminder data, from the famous work of Hans Rosling. 


In [ ]:
fifa = pd.read_csv('data/fifa_player_data.csv.gz', compression='gzip')

In [ ]:
# Let's take a look at the data
fifa.head(5)

In [ ]:
# We can also sort the data in any number of ways
fifa.sort_values('Acceleration', ascending=False).head(5)

In [ ]:
# We can also make the sort work on numerous values
fifa.sort_values(['Acceleration', 'Curve'], ascending=False).head(5)

## Grouping

We can even group items much like we'd do in SQL, perform calculations on some aggregated property, then plot values, just to get an idea of larger scale trends

In [ ]:
fifa.groupby('Nationality').agg({'Ball_Control':'max'}).sort_values('Ball_Control', ascending=False).plot(kind='bar', figsize=(30, 5))

<p style="color:#f1c40f; font-size: 2em">Exercise 1</p>

<p style="background: #f1c40f; padding: 10px; color: #2c3e50">Which <strong>Club_Position</strong> has the players with the highest average <strong>Speed</strong>?</p>

In [ ]:
# Answer 1
fifa.groupby('Club_Position').agg({'Speed':'mean'}).sort_values('Speed', ascending=False).plot(kind='bar', figsize=(30, 5))

<p style="background: #f1c40f; padding: 10px; color: #2c3e50">Which <strong>Club_Position</strong> has the players with the highest average <strong>Speed</strong> variance?</p>

In [ ]:
# Answer 2
fifa.groupby('Club_Position').agg({'Speed':'std'}).sort_values('Speed', ascending=False).plot(kind='bar', figsize=(30, 5))

<p style="background: #f1c40f; padding: 10px; color: #2c3e50">Which <strong>Club</strong> has the players with the highest average <strong>Speed</strong>?</p>

In [ ]:
# Answer 3
fifa.groupby('Club').agg({'Speed':'mean'}).sort_values('Speed', ascending=False).head(10).plot(kind='bar', figsize=(10, 5), ylim=(70,75))

### Histograms

In [ ]:
# We can plot the distributions for all of our features, just to see how everything looks
fifa.hist(bins=20, figsize=(20,20));

In [ ]:
# We can plot 
fifa.Penalties.plot(kind='hist', bins=20)

In [ ]:
fig = plt.figure(figsize=(30, 7))
sns.countplot('Club_Position', data=fifa, palette=sns.color_palette(['#2ecc71']))

This data is a at first glance at little strange. Unfortunately, in FIFA, many players in a squad will not be in the first team. Therefore we have many players classified as 'Sub' or 'Res'. We also have some CF (Centre Forwards) but this is the same as a 'ST' (Striker). 

** Many of our analyses may not care about the position, but in the ones that do, we should ideally clean our data. **

In [ ]:
fig = plt.figure(figsize=(30, 7))
g = sns.countplot('Nationality', data=fifa, palette=sns.color_palette(['#2ecc71']), order=fifa.groupby('Nationality').size().sort_values(ascending=False).index)
# This rotates the labels, otherwise, we can't see anything!
plt.setp(g.get_xticklabels(), rotation=90)
g.figure.get_axes()[0].set_yscale('log')

In [ ]:
# seaborn also offers not just a histogram but also an kernel density enstimation
sns.distplot(fifa.Rating,bins=50, color='#2ecc71')

<p style="color:#f1c40f; font-size: 2em">Exercise 2</p>

<p style="background: #f1c40f; padding: 10px; color: #2c3e50">EASY: Plot the distributions for <strong>Acceleration</strong>.</p>

In [ ]:
# Answer 4
sns.distplot(fifa.Acceleration,bins=50, color='#2ecc71')

<p style="background: #f1c40f; padding: 10px; color: #2c3e50">ADVANCED: Plot two histograms, one on top of the other for <strong>Acceleration</strong> of Italian Players vs Brazilian players. Hint, you'll need to look at subplots in matplotlib, and passing axes in seaborn.</p>

In [ ]:
# Answer 5
fig, ax = plt.subplots(ncols=2, figsize=(20,6))
sns.distplot(fifa.query('Nationality == "Italy"').Rating,bins=50, color='#2ecc71', ax=ax[0])
sns.distplot(fifa.query('Nationality == "Brazil"').Reactions,bins=50, color='red', ax=ax[0])

sns.distplot(fifa.query('Nationality == "Italy"').Rating,bins=20, color='#2ecc71', ax=ax[1])
sns.distplot(fifa.query('Nationality == "Brazil"').Reactions,bins=20, color='red', ax=ax[1])

### Box Plots

In [ ]:
#group by continent
fifa.boxplot('Rating', by='Nationality', rot=90, figsize=(70, 10));

In [ ]:
# Or, we can use seaborn, which has a nicer styling by default.
fig = plt.figure(figsize=(30, 7))
g = sns.boxplot(x="Nationality", y="Acceleration", data=fifa, palette=sns.color_palette(['#2ecc71']))
plt.setp(g.get_xticklabels(), rotation=90);

### Violin Plots

In [ ]:
# Violin plots do a better job of showing us the number of support points across the distribution
fig = plt.figure(figsize=(30, 7))
g = sns.violinplot(x="Club_Position", y="Acceleration", data=fifa, palette=sns.color_palette(['#2ecc71']))

### Swarm Plots

These plots the actual points, with some jitter to help us see how many points (roughly are around particular values).

In [ ]:
# Violin plots do a better job of showing us the number of support points across the distribution
fig = plt.figure(figsize=(30, 7))
g = sns.swarmplot(x="Club_Position", y="Acceleration", data=fifa.sample(1000), palette=sns.color_palette(['#2ecc71']))

In [ ]:
# Swarm Plots can also be overlaid on top of other plots.
fig = plt.figure(figsize=(30, 7))
# We use a sample here, otherwise there are many points to plot.
_sample = fifa.sample(2000)
g = sns.boxplot(x="Nationality", y="Acceleration", data=_sample, palette=sns.color_palette(['#2ecc71']))
g = sns.swarmplot(x="Club_Position", y="Acceleration", data=_sample, palette=sns.color_palette(['#2ecc71']))
plt.setp(g.get_xticklabels(), rotation=90);

### Boxen Plots

In [ ]:
# Violin plots do a better job of showing us the number of support points across the distribution
fig = plt.figure(figsize=(30, 7))
g = sns.boxenplot(x="Club_Position", y="Acceleration", data=fifa, palette=sns.color_palette(['#2ecc71']))

<p style="color:#f1c40f; font-size: 2em">Exercise 3</p>

<p style="background: #f1c40f; padding: 10px; color: #2c3e50">Easy: Create a plot grouped by Club_Position showing the distribution of acceleration across each Nationality?</p>


In [ ]:
# Answer 6a
fig = plt.figure(figsize=(30, 7))
g = sns.boxenplot(x="Nationality", y="Acceleration", hue="Club_Position", data=fifa[fifa.Nationality.isin(['Germany', 'Sweden', 'France', 'Switzerland', 'Israel'])], palette='viridis')
plt.setp(g.get_xticklabels(), rotation=90);

<p style="background: #f1c40f; padding: 10px; color: #2c3e50">ADVANCED: How can we sort the values of the box plots so that they are ordered by mean?</p>

In [ ]:
# Answer 6
fig = plt.figure(figsize=(30, 7))
g = sns.boxplot(x="Nationality", y="Acceleration", data=fifa, palette=sns.color_palette(['#2ecc71']), 
                order=fifa.groupby('Nationality').agg({'Acceleration': ['mean', 'std', 'max', 'min']})
                .sort_values([('Acceleration', 'mean'), ('Acceleration','std'), ('Acceleration','max'), ('Acceleration','min')], ascending=True).index)
plt.setp(g.get_xticklabels(), rotation=90);

## Plotting Joint Distributions

We can also look at the correlations between variables in the traditional scatter plot. We can create them in a number of ways.

### Scatter Plots

In [ ]:
# The first way is using Matplotlib
plt.figure(figsize=(20,6))
plt.scatter(x=fifa['Acceleration'],y=fifa['Speed'], alpha=0.2)
plt.title('Acceleration vs Speed')
plt.xlabel('Acceleration')
plt.ylabel('Speed')

We can also create one image with many plots inside. Here we'll create a number of 

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(10,10))

# We can access each subplot via a 2D Axes Array.
# An easier way in which to access the axes is to use axes.ravel() which will turn the 2D array into a 1D array
# and allow you to reference the first plot with axes[0], second with axes[1] and so on.

axes[0,0].scatter(x=fifa['Acceleration'],y=fifa['Speed'])
axes[0,0].title.set_text('Acceleration vs Speed')
axes[0,0].set_ylabel('Speed')
axes[0,0].set_xlabel('Acceleration')

axes[0,1].scatter(x=fifa['Acceleration'],y=fifa['Age'])
axes[0,1].title.set_text('Acceleration vs Age')
axes[0,1].set_xlabel('Acceleration')
axes[0,1].set_ylabel('Age')

axes[1,1].scatter(x=fifa['Acceleration'],y=fifa['Balance'])
axes[1,1].title.set_text('Acceleration vs Balance')
axes[1,1].set_xlabel('Acceleration')
axes[1,1].set_ylabel('Balance')


axes[1,0].scatter(x=fifa['Acceleration'],y=fifa['Stamina'])
axes[1,0].title.set_text('Acceleration vs Stamina')
axes[1,0].set_xlabel('Acceleration')
axes[1,0].set_ylabel('Stamina')

plt.tight_layout()
plt.show()

In [ ]:
# We can also change the points by hue
plt.figure(figsize=(10,10))
g = sns.scatterplot("Ball_Control", "Dribbling", hue='Club_Position', size='Rating', data=fifa, color="#555555")

### Joint Plots

Seaborn offers a lot of functionality out of the box.
For instance, the joint plot gives us both histograms for the distributions of each variable, and also the joint distribution in the form of a scatter plot.

In [ ]:
g = sns.jointplot("Ball_Control", "Dribbling", data=fifa, kind="reg", color="#555555", height=7)

<p style="color:#f1c40f; font-size: 2em">Exercise 4</p>

<br/>
<div style="background: #f1c40f; padding: 10px; color: #2c3e50"><p>EASY: Which features are more correlated? </p>
<ul>
<li><strong>Reactions</strong> and <strong>Speed</strong>;</li>
<li><strong>Reactions</strong> and <strong>Curve</strong>; or</li>
<li><strong>Reactions</strong> and <strong>Composure</strong></li>
</ul>
</div>

In [ ]:
# Answer 7
sns.jointplot("Speed", "Reactions", data=fifa, kind="reg", color="#555555")
sns.jointplot("Curve", "Reactions", data=fifa, kind="reg", color="#555555")
sns.jointplot("Composure", "Reactions", data=fifa, kind="reg", color="#555555")

Reactions and composure clearly have a stronger correlation here.

But we'll use some computational ways to measure correlation a bit later. For this type of task, it's better to start off with the computation, then look at a small subset of the results.

## Multivariate Distributions

So far, we've seen how we can we can visualize one or two variables and their distributions, but we often have many more variables. Therefore we'd like to be able to visualize more at once to discover areas of interest.

### Scatter Plot Matrices

In [ ]:
# We can do this pretty easily with Seaborn, however, for very large data frames, 
# this is computationally expensive to plot. So we'll focus on one country, Spain!
sns.pairplot(fifa[fifa.Nationality=='Spain'][['Composure', 'Long_Shots', 
                                              'Curve', 'Volleys', 'Jumping', 
                                              'Heading', 'Club_Position']], hue='Club_Position')

<p style="color:#f1c40f; font-size: 2em">Exercise 5</p>

<br/>
<div style="background: #f1c40f; padding: 10px; color: #2c3e50"><p>EASY: This is a great example of a potentially nice visualization can be messed up by a terrible colour map.<br/>
How would you go about improving this visualization?</p>
</div>

In [ ]:
# Answer 8

# The easiest way is to reduce the amount of data we show.
sns.pairplot(fifa[(fifa.Nationality=='Spain') & (fifa.Club_Position.isin(['CB', 'LCB', 'RCB']))][['Composure', 'Long_Shots', 'Curve', 'Volleys', 'Jumping', 
                                'Heading', 'Club_Position']], hue='Club_Position', 
                                 palette=sns.color_palette(['#1abc9c', '#9b59b6', 
                                                            '#34495e', '#bdc3c7']))

In [ ]:
fifa.Club_Position.unique()

In [ ]:
position_mappings= {
    'LW': 'MID',
    'RW': 'MID',
    'ST': 'ATT',
    'GK': 'GK',
    'RCM': 'MID',
    'CAM': 'MID',
    'LCB': 'DEF',
    'LCM': 'MID',
    'RS': 'ATT',
    'RB': 'DEF',
    'RCB': 'DEF',
    'LM': 'MID',
    'LDM': 'MID',
    'RM': 'MID',
    'LB': 'DEF',
    'CDM': 'MID',
    'RDM': 'MID',
    'LF': 'ATT',
    'CB': 'DEF',
    'LAM': 'MID',
    'CM': 'MID',
    'LS': 'ATT',
    'RF': 'ATT',
    'RWB': 'DEF',
    'RAM': 'MID',
    'CF': 'ATT',
}

fifa['Clean_Position'] = fifa.Club_Position.apply(lambda x: position_mappings[x] if x in position_mappings else np.nan)

_clean = fifa[~fifa.Clean_Position.isnull()]
sns.pairplot(_clean[(_clean.Nationality=='Spain')][['Composure', 'Long_Shots', 'Curve', 'Volleys', 'Jumping', 
                                'Heading', 'Clean_Position']], hue='Clean_Position')

In [ ]:
# The harder way is to clean our data and group club positions in to goal keepers, defenders, midfielders, and strikers - less groups means less colours.


Plotting individual points is cool, but there is a lot of noise in these plots. There are other techniques you can use to increase the saliency of content in the visualization. 

A key question to always ask yourself is, what do I really want to see? In this instance, I want to see which values are more correlated with each other. Therefore having all the points, while nice, will slow down my visualization, and add noise.

So, with that in mind, let's look at visualizing this data in a more clean way using Kernel Density Estimations.

### KDE Matrices

In [ ]:
g = sns.PairGrid(fifa[fifa.Nationality=='Spain'][['Composure', 'Long_Shots', 'Curve', 'Volleys', 'Jumping', 'Heading']])
g.map_diag(sns.kdeplot)
g.map_offdiag(sns.kdeplot, cmap="Blues_d", n_levels=6);

<div style="background: #f1c40f; padding: 10px; color: #2c3e50"><p>EASY: Switch out my variables for some that you are interested in.<br/></p>
</div>

In [ ]:
# Answer 9

### Parallel Coordinates

My favourite! Perfect for visualizing many variables at once.

In [ ]:
spanish_players = fifa[(fifa.Nationality  == 'Spain') & (fifa.Club_Position.isin(['RF', 'CB', 'GK']))]
string_columns = [fifa.columns[idx] for idx, data_type in enumerate(fifa.dtypes) if data_type == 'object']
string_columns.pop(string_columns.index('Club_Position'))

In [ ]:
plt.figure(figsize=(30, 6))
g = parallel_coordinates(spanish_players.drop(string_columns + ['Contract_Expiry'], axis=1), 'Club_Position', colormap='viridis')
plt.legend(loc=(1,1))
plt.setp(g.get_xticklabels(), rotation=90);

You can play with these if you want :D

### RadVis 

Sometimes confusing, but I think they are neat!

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=2, figsize = (20,6))

plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['axes.edgecolor'] = '#ecf0f1'

countries = [['Italy', 'Spain'], ['England', 'Russia']]
positions = ['RM', 'LM', 'CAM']
n_rows = 2
n_cols= 2

for row in range(n_rows):
    for col in range(n_cols):
        ax = axes[row, col]
        country=countries[row][col]

        players_no_gk_sub_res = fifa[(fifa.Nationality  == country) & (fifa.Club_Position.isin(positions))]
        radviz(players_no_gk_sub_res[['Shot_Power', 'Agility', 'Speed', 'Acceleration', 'Vision', 'Ball_Control', 'Club_Position']], 'Club_Position', colormap='viridis', ax=ax)
        
        ax.set_title(country)
        
#         if(row != 0 & col !=1):
#             legend = ax.legend()
#             legend.remove()

## Global Correlation Detection

While the techniques we've shown above are good for a few data points, they don't scale so well. What we often do instead is go for a global view of the data first in order to understand the relationships at a macro level.

This is a very fast process to carry out in Pandas.

In [ ]:
# Correlation Tables for all variables can be quickly computed across a large dataframe
fifa.corr()

<p style="color:#f1c40f; font-size: 2em">Exercise 6</p>

<p style="background: #f1c40f; padding: 10px; color: #2c3e50">MEDIUM: So, you can quickly calculate all feature correlations using Pandas, but reading all these numbers is a cognitively demanding task. How can we visualize such data? Hint, have a look at the seaborn gallery :)</p>

In [ ]:
# Answer 10
plt.figure(figsize=(10,10))
sns.heatmap(fifa.corr(), cmap='PuOr_r')

<p style="background: #f1c40f; padding: 10px; color: #2c3e50">MEDIUM: Remember in our lectures how we showed it was easier for people to compare heatmaps when related information is place together? <br/>What component in seaborn could help you do this?<br/>Can you find the surprising relationship using this plot?</p>

In [ ]:
# Answer 11
g = sns.clustermap(fifa.corr(), cmap='PuOr_r')
plt.setp(g.ax_heatmap.get_yticklabels(), rotation=0);

> You're almost on the cusp of data science now :D

<p style="background: #f1c40f; padding: 10px; color: #2c3e50">ADVANCED: At the macro level, we always lose information, since we lose granularity. Can you partition the data on some level to see how correlations change?</p>

In [ ]:
# Answer 12
# Hint, you can for instance, query the dataframe by some Nationalities, and plot each correlation heatmap in 
# a separate plot.
fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(20,7))
sns.heatmap(fifa[fifa.Nationality == 'Spain'].corr(), ax=axes[0])
sns.heatmap(fifa[fifa.Nationality == 'Algeria'].corr(), ax=axes[1])

## Faceted Visualizations

How you split the data can also provide many insights, and there are many ways in which we could split this data (which is why I chose it :)).

In [ ]:
# or with linear regression
g = sns.FacetGrid(fifa, col="Club_Position", size=2, aspect=1, col_wrap=5, sharex=True, sharey=False)
g.map(plt.hist, 'Acceleration', bins=20).add_legend()

> Cool, right? You can play with this more

In [ ]:
g = sns.FacetGrid(fifa[fifa.Nationality.isin(['Spain', 'Italy', 'England', 'France'])], col="Club_Position", 
                  hue="Nationality", size=2, aspect=1, col_wrap=5, sharex=True, sharey=False)
g.map(plt.scatter, 'Acceleration', 'Ball_Control').add_legend()

## Building a Scouting System

** Clustering with Seaborn, with scipy in the back office) **

Seaborn also has some great tools available to help in some common tasks such as data clustering.

With this in mind, we'll have a look at how we can use seaborn to scout players to find cheaper alternatives for the expensive ones :)

In [ ]:
# Pivot tables allow us to change how our data is formatted.
# Here I want to have a row per player, where I have 3 features
pivoted = fifa[fifa.Club_Position == 'ST'].pivot_table(index=['Name'], values=['Ball_Control', 'Reactions', 'Strength', 'Acceleration'])

In [ ]:
pivoted.head()

In [ ]:
g = sns.clustermap(pivoted, z_score=1, figsize=(10,70), cmap='PuOr_r')
plt.setp(g.ax_heatmap.get_yticklabels(), rotation=0);

In [ ]:
# We'll remove the string columns, since Parallel Coordinates etc. will not work with them
string_columns = [fifa.columns[idx] for idx, data_type in enumerate(fifa.dtypes) if data_type == 'object']
string_columns.pop(string_columns.index('Name'))
filtered_data = fifa[fifa.Name.isin(['Romelu Lukaku', 'Artem Dzyuba', 'Diego Costa', 'Christian Benteke', 'Negredo', 'Nino', 'Alexandre Lacazette'])].drop(string_columns + ['Contract_Expiry'], axis=1)

In [ ]:
fig = plt.figure(figsize=(30, 7))
pc = parallel_coordinates(filtered_data, 'Name', colormap='viridis')
plt.setp(pc.get_xticklabels(), rotation=90);

In [ ]:
plt.figure(figsize=(10, 6))
radviz(filtered_data[['Stamina', 'Agility', 'Speed', 'Acceleration', 'Strength', 'Ball_Control', 'Aggression', 'Name']], 'Name',colormap='viridis')
plt.legend(loc=(1,0.6));

## Styling Matplotlib Plots

By default, matplotlib is undoubtedly ugly. But, all is not lost, you can clean up matplotlib plots 

In [ ]:
pd.set_option("display.max_columns",None)
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['axes.edgecolor'] = '#ecf0f1'
plt.rcParams['grid.color'] = '#ecf0f1'
plt.rcParams['axes.labelcolor'] = '#7f8c8d'
plt.rcParams['text.color'] = '#7f8c8d'

In [ ]:
plt.figure(figsize=(10, 6))
radviz(filtered_data[['Stamina', 'Agility', 'Speed', 'Acceleration', 'Strength', 'Ball_Control', 'Aggression', 'Name']], 'Name', colormap='Spectral')
plt.legend(loc=(1,0.6)) 

> Wow, I hear you say, how beautiful :D

## Your Own Exploratory Analysis

<p style="color:#f1c40f; font-size: 2em">Exercise 7</p>

<p style="background: #f1c40f; padding: 10px; color: #2c3e50">OPTIONAL: Conduct your own interesting analysis.</p>